In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [ ]:
!apt-get update
!apt-get install -y unzip xvfb libxi6 libgconf-2-4
!apt-get -f install
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg --install google-chrome-stable_current_amd64.deb
!apt-get -f install
!dpkg --install google-chrome-stable_current_amd64.deb
!wget https://chromedriver.storage.googleapis.com/129.0.3955.36/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/


Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:8 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,090 B]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,915 B in 2s (1,803 B/s)
Reading package lists... Done
W: Skipping

In [ ]:
!pip install selenium


In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver
driver = webdriver.Chrome(options=chrome_options)

wait = WebDriverWait(driver, 10)

# handle cookie popup and others
def handle_popups():
    try:
        allow_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')))
        allow_cookies_button.click()
        print("Accepted cookies.")
    except Exception as e:
        print(f"Could not find or accept cookies: {e}")

    try:
        newsletter_popup = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'newsletter-popup-close')))
        newsletter_popup.click()
        print("Closed newsletter popup.")
    except Exception as e:
        print("No newsletter popup found.")

# cick vis flere btn
def load_all_articles():
    while True:
        try:
            show_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Vis flere')]")))
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)
            print("Clicked 'Vis flere' to load more articles.")
        except Exception as e:
            print(f"No more 'Vis flere' button found or error occurred: {e}")
            break

def login():
    try:
        login_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'sticky-header__hamburger-profil-login')))
        driver.execute_script("arguments[0].click();", login_button)
        print("Login button clicked.")
    except Exception as e:
        print(f"Error clicking login button: {e}")

    time.sleep(2)

    try:
        email_input = wait.until(EC.visibility_of_element_located((By.ID, 'email')))
        password_input = driver.find_element(By.ID, 'current-password')

        email_input.clear()
        email_input.send_keys('grp613@alumni.ku.dk')

        password_input.clear()
        password_input.send_keys('xxxx')
        print("Filled in email and password using send_keys.")

        login_submit_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        login_submit_button.click()
        print("Login form submitted.")

        time.sleep(5)

        driver.save_screenshot('login_status_after_sendkeys.png')

        try:
            profile_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'sp_profile_menu'))
            )
            print("Login successful, profile element found.")
        except Exception as e:
            print(f"Login verification failed: {e}")
            driver.save_screenshot('login_failed_after_sendkeys.png')

    except Exception as e:
        print(f"Error during login: {e}")


base_url = "https://www.berlingske.dk/search?query={term}"

all_articles = []

def load_page_with_retry(url, retries=3):
    for attempt in range(retries):
        try:
            driver.get(url)
            return True
        except Exception as e:
            print(f"Failed to load page: {url} on attempt {attempt + 1}: {e}")
            time.sleep(2)
    return False

# paywall and relogin
def check_paywall_and_relogin():
    try:
        paywall_element = driver.find_element(By.ID, 'paywall-article')
        if paywall_element.is_displayed():
            print("Paywall detected. Re-logging in...")
            login()
            driver.refresh()
            time.sleep(5)
            return True
    except Exception as e:
        print(f"No paywall detected or issue finding paywall: {e}")
        return False

def scrape_berlingske_articles(search_term):
    search_url = base_url.format(term=search_term)
    if not load_page_with_retry(search_url):
        print(f"Failed to load search page for term: {search_term}")
        return

    load_all_articles()

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('div', class_='teaser teaser--border-bottom teaser--search d-flex flex-row w-100')

    if not articles:
        print(f"No articles found for search term: {search_term}.")
        return

    for article in articles:
        article_url = "https://www.berlingske.dk" + article.find('a')['href']
        if not load_page_with_retry(article_url):
            print(f"Failed to load article page: {article_url}")
            continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        driver.execute_script("var banners = document.querySelectorAll('div[id^=banner], #NewsletterSignUp'); banners.forEach(banner => banner.remove());")
        time.sleep(1)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'article-body'))
            )
        except Exception as e:
            print(f"Error waiting for article body to load: {e}")

        article_soup = BeautifulSoup(driver.page_source, 'html.parser')
        for unwanted_element in article_soup.find_all(['div', 'button'], class_=['newsletter-popup', 'NewsletterSignUp', 'embedded-code']):
            unwanted_element.decompose()

        try:
            content = " ".join([p.get_text() for p in article_soup.find_all('p')])
            if not content.strip():
                content = 'No content found in article body'
        except Exception as e:
            print(f"Error extracting content: {e}")
            content = 'No content'

        try:
            title = article_soup.find('h2', class_='article-header__title').get_text()
        except:
            title = 'No title'

        try:
            subheader = article_soup.find('p', class_='article-header__intro').get_text()
        except:
            subheader = 'No subheader'

        try:
            date = article_soup.find('div', class_='article-byline__date').get_text()
        except:
            date = 'No date'

        try:
            category = article_soup.find('a', class_='label__category').get_text()
        except:
            category = 'No category'

        all_articles.append({
            'search_term': search_term,
            'url': article_url,
            'title': title,
            'subheader': subheader,
            'date': date,
            'category': category,
            'content': content.strip()
        })

        print(f"Scraped article for search term '{search_term}'")


search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']


driver.get('https://www.berlingske.dk/')
handle_popups()
login()

for term in search_terms:
    scrape_berlingske_articles(term)



df = pd.DataFrame(all_articles)

Accepted cookies.
No newsletter popup found.
Login button clicked.
Filled in email and password using send_keys.
Login form submitted.
Login successful, profile element found.
Clicked 'Vis flere' to load more articles.
Clicked 'Vis flere' to load more articles.
Clicked 'Vis flere' to load more articles.
No more 'Vis flere' button found or error occurred: Message: 

Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recognition'
Scraped article for search term 'facial recogniti

In [ ]:

df['merged_content'] = df[['title', 'subheader', 'content']].fillna('').agg(' '.join, axis=1)



In [ ]:
import re
import pandas as pd

# fixing date
month_mapping = {
    'januar': 'January', 'februar': 'February', 'marts': 'March',
    'april': 'April', 'maj': 'May', 'juni': 'June',
    'juli': 'July', 'august': 'August', 'september': 'September',
    'oktober': 'October', 'november': 'November', 'december': 'December'
}

def clean_date(date_str):
    cleaned_date = re.sub(r'.*?d\. ', '', date_str)
    cleaned_date = re.split(r'kl\. ', cleaned_date)[0].strip()

    for danish_month, english_month in month_mapping.items():
        cleaned_date = cleaned_date.replace(danish_month, english_month)

    cleaned_date = cleaned_date.replace(',', '').strip()

    return cleaned_date

df['cleaned_date'] = df['date'].apply(clean_date)

df['date'] = pd.to_datetime(df['cleaned_date'], format='%d. %B %Y', dayfirst=True, errors='coerce')

df = df.drop(columns=['cleaned_date'])



In [ ]:
df = df.rename(columns={'title': 'header'})

In [ ]:
df

,search_term,url,header,subheader,date,category,content,merged_content
0,facial recognition,https://www.berlingske.dk/business/business-up...,Business-update: En »særdeles erfaren direktør...,Forsikringsselskaber melder om færre vejrskade...,2024-07-31,\n Business\n,Forsikringsselskaber melder om færre vejrskade...,Business-update: En »særdeles erfaren direktør...
1,facial recognition,https://www.berlingske.dk/internationalt/laekk...,Lækket dokument afslører: EU vil forbyde ansig...,Et midlertidigt forbud mod ansigtsgenkendelse ...,2020-01-17,\n Internationalt\n,Et midlertidigt forbud mod ansigtsgenkendelse ...,Lækket dokument afslører: EU vil forbyde ansig...
2,facial recognition,https://www.berlingske.dk/virksomheder/ansigts...,Ansigtsgenkendelse er selv kommet i søgeren,"Software, der kan identificere folk på billede...",2020-01-02,\n Virksomheder\n,"Software, der kan identificere folk på billede...",Ansigtsgenkendelse er selv kommet i søgeren So...
3,facial recognition,https://www.berlingske.dk/kommentarer/orwellsk...,Orwellske fantasier og brutale diktaturstater ...,At politiet fører tilsyn med offentlige område...,2019-12-27,\n Kommentarer\n,Dette er en kommentar. Den udtrykker skribente...,Orwellske fantasier og brutale diktaturstater ...
4,facial recognition,https://www.berlingske.dk/internationalt/kina-...,Kina er et skridt tættere på at være en »dysto...,Ansigtsgenkendelse er blevet et obligatorisk s...,2019-12-02,\n Internationalt\n,Ansigtsgenkendelse er blevet et obligatorisk s...,Kina er et skridt tættere på at være en »dysto...
...,...,...,...,...,...,...,...,...
621,tryghedsskabende kamera,https://www.berlingske.dk/samfund/nye-detaljer...,Nye detaljer: Amerikaner ville selv dræbe Kurt...,Amerikansk terrorist ville finde våben i Europ...,2015-04-22,\n Indland\n,Amerikansk terrorist ville finde våben i Europ...,Nye detaljer: Amerikaner ville selv dræbe Kurt...
622,tryghedsskabende kamera,https://www.berlingske.dk/samfund/se-laesernes...,Se læsernes lykønskninger til dronningen - og ...,Berlingskes læsere sendte fødselsdagshilsner t...,2015-04-15,\n Indland\n,Berlingskes læsere sendte fødselsdagshilsner t...,Se læsernes lykønskninger til dronningen - og ...
623,tryghedsskabende kamera,https://www.berlingske.dk/kommentarer/big-brot...,Big Brother i dit eget hus,,2013-10-28,\n Kommentarer\n,Dette er en kommentar. Den udtrykker skribente...,Big Brother i dit eget hus Dette er en kommen...
624,tryghedsskabende kamera,https://www.berlingske.dk/samfund/cepos-overva...,CEPOS: Overvågning skal ikke udbredes,"Danskerne er begejstrede for overvågning, men ...",2011-10-20,\n Indland\n,"Danskerne er begejstrede for overvågning, men ...",CEPOS: Overvågning skal ikke udbredes Danskern...


In [ ]:
import re

# cleaning the end of the article that has a story of Berlingske
def remove_after_string(text):
    if isinstance(text, str):
        return re.sub(r'Grundlagt 1749.*', '', text, flags=re.DOTALL).strip()
    return text


df['merged_content_cleaned'] = df['merged_content'].apply(remove_after_string)


print(df[['merged_content', 'merged_content_cleaned']].head())

df = df.drop(columns=['merged_content']).rename(columns={'merged_content_cleaned': 'merged_content'})
df

                                      merged_content  \
0  Business-update: En »særdeles erfaren direktør...   
1  Lækket dokument afslører: EU vil forbyde ansig...   
2  Ansigtsgenkendelse er selv kommet i søgeren So...   
3  Orwellske fantasier og brutale diktaturstater ...   
4  Kina er et skridt tættere på at være en »dysto...   

                              merged_content_cleaned  
0  Business-update: En »særdeles erfaren direktør...  
1  Lækket dokument afslører: EU vil forbyde ansig...  
2  Ansigtsgenkendelse er selv kommet i søgeren So...  
3  Orwellske fantasier og brutale diktaturstater ...  
4  Kina er et skridt tættere på at være en »dysto...  


,search_term,url,header,subheader,date,category,content,merged_content
0,facial recognition,https://www.berlingske.dk/business/business-up...,Business-update: En »særdeles erfaren direktør...,Forsikringsselskaber melder om færre vejrskade...,2024-07-31,\n Business\n,Forsikringsselskaber melder om færre vejrskade...,Business-update: En »særdeles erfaren direktør...
1,facial recognition,https://www.berlingske.dk/internationalt/laekk...,Lækket dokument afslører: EU vil forbyde ansig...,Et midlertidigt forbud mod ansigtsgenkendelse ...,2020-01-17,\n Internationalt\n,Et midlertidigt forbud mod ansigtsgenkendelse ...,Lækket dokument afslører: EU vil forbyde ansig...
2,facial recognition,https://www.berlingske.dk/virksomheder/ansigts...,Ansigtsgenkendelse er selv kommet i søgeren,"Software, der kan identificere folk på billede...",2020-01-02,\n Virksomheder\n,"Software, der kan identificere folk på billede...",Ansigtsgenkendelse er selv kommet i søgeren So...
3,facial recognition,https://www.berlingske.dk/kommentarer/orwellsk...,Orwellske fantasier og brutale diktaturstater ...,At politiet fører tilsyn med offentlige område...,2019-12-27,\n Kommentarer\n,Dette er en kommentar. Den udtrykker skribente...,Orwellske fantasier og brutale diktaturstater ...
4,facial recognition,https://www.berlingske.dk/internationalt/kina-...,Kina er et skridt tættere på at være en »dysto...,Ansigtsgenkendelse er blevet et obligatorisk s...,2019-12-02,\n Internationalt\n,Ansigtsgenkendelse er blevet et obligatorisk s...,Kina er et skridt tættere på at være en »dysto...
...,...,...,...,...,...,...,...,...
621,tryghedsskabende kamera,https://www.berlingske.dk/samfund/nye-detaljer...,Nye detaljer: Amerikaner ville selv dræbe Kurt...,Amerikansk terrorist ville finde våben i Europ...,2015-04-22,\n Indland\n,Amerikansk terrorist ville finde våben i Europ...,Nye detaljer: Amerikaner ville selv dræbe Kurt...
622,tryghedsskabende kamera,https://www.berlingske.dk/samfund/se-laesernes...,Se læsernes lykønskninger til dronningen - og ...,Berlingskes læsere sendte fødselsdagshilsner t...,2015-04-15,\n Indland\n,Berlingskes læsere sendte fødselsdagshilsner t...,Se læsernes lykønskninger til dronningen - og ...
623,tryghedsskabende kamera,https://www.berlingske.dk/kommentarer/big-brot...,Big Brother i dit eget hus,,2013-10-28,\n Kommentarer\n,Dette er en kommentar. Den udtrykker skribente...,Big Brother i dit eget hus Dette er en kommen...
624,tryghedsskabende kamera,https://www.berlingske.dk/samfund/cepos-overva...,CEPOS: Overvågning skal ikke udbredes,"Danskerne er begejstrede for overvågning, men ...",2011-10-20,\n Indland\n,"Danskerne er begejstrede for overvågning, men ...",CEPOS: Overvågning skal ikke udbredes Danskern...


In [ ]:
from google.colab import files

df.to_csv('Berlingske.csv', index=False)
files.download('Berlingske.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df['merged_content'][0]

'Business-update: En »særdeles erfaren direktør« for københavnsk advokatkontor er tiltalt for skattesvindel Forsikringsselskaber melder om færre vejrskader trods regn, og københavnsk advokat er tiltalt for skattesvindel. Det er nogle af overskrifterne i dagens update fra Berlingske Business. Forsikringsselskaber melder om færre vejrskader trods regn, og københavnsk advokat er tiltalt for skattesvindel. Det er nogle af overskrifterne i dagens update fra Berlingske Business. \n                                                                            Olivia Emma Espersen\n                                                                     Journalist  Godeftermiddag og velkommen til dagens Business-update her på Berlingske. Vi giver dig et overblik over dagens indtil videre vigtigste erhvervsnyheder. God læselyst! Der har foreløbigt været flere dage med regnvejr end solskin denne sommer. Ifølge DMI har det været den vådeste sommer i\xa0Danmark siden 1874. Men dét har ikke ledt til en st

In [ ]:
from google.colab import files

df.to_csv('Berlinske_scraped_articles_all.csv', index=False)
files.download('Berlinske_scraped_articles_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>